# 1. Information about the submission

## 1.1 Name and number of the assignment 

### Text categorization: argument mining. Assignment 2.

## 1.2 Student name

### Nuzhnov Mark

## 1.3 Codalab user ID

### Nuzhnov_Mark

## 1.4 Additional comments

***Enter here** any additional comments which you would like to communicate to a TA who is going to grade this work not related to the content of your submission.*

# 2. Technical Report

## 2.1 Methodology 


BERT (Bidirectional Encoder Representations from Transformers) is a recent paper published by researchers at Google AI Language. In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix $Q$.   The keys and values are also packed together into matrices $K$ and $V$.  We compute the matrix of outputs as:                      
                                                                 
$$                                                                         
   \mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V               
$$   

![alt text](https://lena-voita.github.io/resources/lectures/seq2seq/transformer/qkv_explained-min.png)
Source: [Lena Voita's Lecture about Seq2Seq](https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html)

Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.                                            
$$    
\mathrm{MultiHead}(Q, K, V) = \mathrm{Concat}(\mathrm{head_1}, ..., \mathrm{head_h})W^O    \\                                           
    \text{where}~\mathrm{head_i} = \mathrm{Attention}(QW^Q_i, KW^K_i, VW^V_i)                                
$$                                                                                                                 

Where the projections are parameter matrices $W^Q_i \in \mathbb{R}^{d_{\text{model}} \times d_k}$, $W^K_i \in \mathbb{R}^{d_{\text{model}} \times d_k}$, $W^V_i \in \mathbb{R}^{d_{\text{model}} \times d_v}$ and $W^O \in \mathbb{R}^{hd_v \times d_{\text{model}}}$.                                                                                                                                                                                             In this work I tried employ $h=3$, $h=5$, $h=8$ parallel attention layers, or heads. For each of these I use $d_k=d_v=d_{\text{model}}/h=64$. Due to the reduced dimension of each head, the total computational cost is similar to that of single-head attention with full dimensionality. 

![alt text](https://jalammar.github.io/images/t/transformer_multi-headed_self-attention-recap.png)
Source: [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)

## 2.2 Discussion of results

When I ran baseline on Quarantine I got pretty bad results so I decided not to train it further and add MHA. First I tried 6 heads with key_dim=128 but results on dev were poor because I probably overfitted. Then I switched to 3 heads with key_dim=64. I was able to achieve f1-stance 0.5044 and f1-premise 0.4405 on development and 0.5384 and 0.4948 on post-eval.

Training process details are available on [wandb](https://wandb.ai/darkdestiny/Argument%20mining?workspace=user-benchm)

# 3. Code

*Enter here all code used to produce your results submitted to Codalab. Add some comments and subsections to navigate though your solution.*

*In this part you are expected to develop yourself a solution of the task and provide a reproducible code:*
- *Using Python 3;*
- *Contains code for installation of all dependencies;*
- *Contains code for downloading of all the datasets used*;
- *Contains the code for reproducing your results (in other words, if a tester downloads your notebook she should be able to run cell-by-cell the code and obtain your experimental results as described in the methodology section)*.


*As a result, you code will be graded according to these criteria:*
- ***Readability**: your code should be well-structured preferably with indicated parts of your approach (Preprocessing, Model training, Evaluation, etc.).*
- ***Reproducibility**: your code should be reproduced without any mistakes with “Run all” mode (obtaining experimental part).*


## 3.1 Requirements

In [1]:
! pip install transformers
! pip install gdown
! pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=22d6cc86c80

In [2]:
!wandb login 7ed66683ab734c5607259a94b5937616a911f67b

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import MultiHeadAttention, Flatten, TimeDistributed
from tensorflow.keras.optimizers.experimental import AdamW
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import numpy as np
from sklearn.metrics import classification_report
import wandb
from wandb.keras import WandbCallback
#from google.colab import drive
#drive.mount('/content/drive')

seed_value = 42
os.environ['PYTHONHASHSEED']=str(seed_value)

In [4]:
# Name of the BERT model to use
model_name = 'DeepPavlov/rubert-base-cased-sentence'

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config=config)

# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config=config, from_pt=True)

# Load the MainLayer
bert = transformer_model.layers[0]

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


## 3.2 Download the data

In [11]:
!gdown 19LzgUlM3417TlG6bmo-eSY5vRAKw2ukR
!gdown 1eu97ydTza6HGyvsD3nz_mp-bFgxN9mCv

Downloading...
From: https://drive.google.com/uc?id=19LzgUlM3417TlG6bmo-eSY5vRAKw2ukR
To: /content/train_all.tsv
100% 1.54M/1.54M [00:00<00:00, 160MB/s]
Downloading...
From: https://drive.google.com/uc?id=1eu97ydTza6HGyvsD3nz_mp-bFgxN9mCv
To: /content/test-no_labels.tsv
100% 298k/298k [00:00<00:00, 130MB/s]


## 3.3 Quarantine

### 3.3.1 Preprocessing

In [ ]:
CLASS_NAME = "quarantine"
# Import data from csv
whole_data = pd.read_csv('./train_all.tsv', sep='\t')

# Train_test_split
test_size = 0.2
data, data_test = train_test_split(whole_data, test_size=test_size, random_state = seed_value)


#------------------------------------------------------------------------------------#
## Train
# Select required columns
data = data[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data['stance_label'] = pd.Categorical(data[f'{CLASS_NAME}_stance'])
data['argument_label'] = pd.Categorical(data[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data[f'{CLASS_NAME}_stance'] = data['stance_label'].cat.codes
data[f'{CLASS_NAME}_argument'] = data['argument_label'].cat.codes

#------------------------------------------------------------------------------------#
## Test
# Select required columns
data_test = data_test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data_test['stance_label'] = pd.Categorical(data_test[f'{CLASS_NAME}_stance'])
data_test['argument_label'] = pd.Categorical(data_test[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data_test[f'{CLASS_NAME}_stance'] = data_test['stance_label'].cat.codes
data_test[f'{CLASS_NAME}_argument'] = data_test['argument_label'].cat.codes

In [ ]:
# Ready output data for the model
test_y_stance = to_categorical(data_test[f'{CLASS_NAME}_stance'])
test_y_argument = to_categorical(data_test[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
test_x = tokenizer(
    text=data_test['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

### 3.3.2 Architecture

In [ ]:
# Build your model
input_ids = Input(shape=(256,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(256,), name='attention_mask', dtype='int32')
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[0]

# Add multi-head attention layer
attention_output = MultiHeadAttention(num_heads=3, key_dim=64)(bert_model, bert_model)

# Flatten the output from multi-head attention layer
flatten = Flatten()(attention_output)

# Apply dropout layer
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(flatten, training=False)

# Then build your model output
stance = Dense(units=len(data.stance_label.value_counts()), activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='stance')(pooled_output)
argument = Dense(units=len(data.argument_label.value_counts()), activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='argument')(pooled_output)
outputs = {'stance': stance, 'argument': argument}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  177853440   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 256,                      

### 3.3.3 Training

In [ ]:
# Set an optimizer
optimizer = AdamW(
    learning_rate=1e-6,
    epsilon=1e-08,
    weight_decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'stance': CategoricalCrossentropy(from_logits = True), 'argument': CategoricalCrossentropy(from_logits = True)}
metric = {'stance': CategoricalAccuracy('accuracy'), 'argument': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Ready output data for the model
y_stance = to_categorical(data[f'{CLASS_NAME}_stance'])
y_argument = to_categorical(data[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

wandb.init(project="Argument mining", name = "Bert_attention_quarantine", tags = ["Ruberta_MHA", "RB"])
epochs = 8
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    y={'stance': y_stance, 'argument': y_argument},
    validation_data=({'input_ids': test_x['input_ids'][:8], 'attention_mask': test_x['attention_mask'][:8]}, 
                     {'stance': test_y_stance[:8], 'argument': test_y_argument[:8]}),
    batch_size=8,
    epochs=epochs, callbacks=[WandbCallback()])
wandb.finish()

Epoch 1/8


672/672 [==============================] - ETA: 0s - loss: 0.9818 - argument_loss: 0.4532 - stance_loss: 0.5286 - argument_accuracy: 0.8608 - stance_accuracy: 0.8115

wandb: Adding directory to artifact (/content/wandb/run-20230425_134736-xmwd6zns/files/model-best)... Done. 17.3s


672/672 [==============================] - 497s 626ms/step - loss: 0.9818 - argument_loss: 0.4532 - stance_loss: 0.5286 - argument_accuracy: 0.8608 - stance_accuracy: 0.8115 - val_loss: 0.1056 - val_argument_loss: 0.0269 - val_stance_loss: 0.0787 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000
Epoch 2/8
672/672 [==============================] - 330s 491ms/step - loss: 0.5390 - argument_loss: 0.2343 - stance_loss: 0.3047 - argument_accuracy: 0.9390 - stance_accuracy: 0.8867 - val_loss: 0.1387 - val_argument_loss: 0.0393 - val_stance_loss: 0.0994 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000
Epoch 3/8
672/672 [==============================] - ETA: 0s - loss: 0.4763 - argument_loss: 0.2003 - stance_loss: 0.2760 - argument_accuracy: 0.9412 - stance_accuracy: 0.8958

wandb: Adding directory to artifact (/content/wandb/run-20230425_134736-xmwd6zns/files/model-best)... Done. 18.1s


...

wandb: Adding directory to artifact (/content/wandb/run-20230425_134736-xmwd6zns/files/model-best)... Done. 17.8s


672/672 [==============================] - 406s 604ms/step - loss: 0.4071 - argument_loss: 0.1703 - stance_loss: 0.2368 - argument_accuracy: 0.9468 - stance_accuracy: 0.9112 - val_loss: 0.0597 - val_argument_loss: 0.0176 - val_stance_loss: 0.0421 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000
Epoch 6/8
672/672 [==============================] - 329s 489ms/step - loss: 0.3690 - argument_loss: 0.1557 - stance_loss: 0.2134 - argument_accuracy: 0.9511 - stance_accuracy: 0.9207 - val_loss: 0.1474 - val_argument_loss: 0.0742 - val_stance_loss: 0.0732 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000
Epoch 7/8
672/672 [==============================] - ETA: 0s - loss: 0.3368 - argument_loss: 0.1391 - stance_loss: 0.1978 - argument_accuracy: 0.9576 - stance_accuracy: 0.9302

wandb: Adding directory to artifact (/content/wandb/run-20230425_134736-xmwd6zns/files/model-best)... Done. 18.2s


672/672 [==============================] - 415s 618ms/step - loss: 0.3368 - argument_loss: 0.1391 - stance_loss: 0.1978 - argument_accuracy: 0.9576 - stance_accuracy: 0.9302 - val_loss: 0.0506 - val_argument_loss: 0.0282 - val_stance_loss: 0.0224 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000
Epoch 8/8
672/672 [==============================] - 328s 489ms/step - loss: 0.3119 - argument_loss: 0.1322 - stance_loss: 0.1797 - argument_accuracy: 0.9570 - stance_accuracy: 0.9304 - val_loss: 0.0672 - val_argument_loss: 0.0325 - val_stance_loss: 0.0347 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000


argument_accuracy,▁▇▇▇▇███
argument_loss,█▃▂▂▂▂▁▁
epoch,▁▂▃▄▅▆▇█
loss,█▃▃▂▂▂▁▁
stance_accuracy,▁▅▆▇▇▇██
stance_loss,█▄▃▂▂▂▁▁
val_argument_accuracy,▁▁▁▁▁▁▁▁
val_argument_loss,▂▄▁▂▁█▂▃
val_loss,▅▇▃▃▂█▁▂
val_stance_accuracy,▁▁▁▁▁▁▁▁
val_stance_loss,▆█▄▄▃▆▁▂


### 3.3.4 Prediction

In [ ]:
test = pd.read_csv("./val_empty.tsv", sep='\t')
test.head()
test_d = test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]
for_pred = tokenizer(
    text=test_d['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
test_results = model.predict(x={'input_ids': for_pred['input_ids'], 'attention_mask': for_pred['attention_mask']})

45/45 [==============================] - 28s 547ms/step


In [ ]:
test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1) - 1
test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1) - 1
test_d[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']].to_csv(f"./val_predict_MHA_{CLASS_NAME}.tsv", sep='\t', index=None)

<ipython-input-50-c8ea65027b6b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1) - 1
<ipython-input-50-c8ea65027b6b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1) - 1


In [ ]:
test = pd.read_csv("./test-no_labels.tsv", sep='\t')
test.head()
test_d = test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]
for_pred = tokenizer(
    text=test_d['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
test_results = model.predict(x={'input_ids': for_pred['input_ids'], 'attention_mask': for_pred['attention_mask']})

44/44 [==============================] - 24s 548ms/step


In [ ]:
test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1) - 1
test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1) - 1
test_d[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']].to_csv(f"./test_predict_MHA_{CLASS_NAME}.tsv", sep='\t', index=None)

<ipython-input-52-3d4f37872f57>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1) - 1
<ipython-input-52-3d4f37872f57>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1) - 1


## 3.4 Masks

### 3.4.1 Preprocessing

In [6]:
CLASS_NAME = "masks"
# Import data from csv
whole_data = pd.read_csv('./train_all.tsv', sep='\t')

# Train_test_split
test_size = 0.2
data, data_test = train_test_split(whole_data, test_size=test_size, random_state = seed_value)


#------------------------------------------------------------------------------------#
## Train
# Select required columns
data = data[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data['stance_label'] = pd.Categorical(data[f'{CLASS_NAME}_stance'])
data['argument_label'] = pd.Categorical(data[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data[f'{CLASS_NAME}_stance'] = data['stance_label'].cat.codes
data[f'{CLASS_NAME}_argument'] = data['argument_label'].cat.codes

#------------------------------------------------------------------------------------#
## Test
# Select required columns
data_test = data_test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data_test['stance_label'] = pd.Categorical(data_test[f'{CLASS_NAME}_stance'])
data_test['argument_label'] = pd.Categorical(data_test[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data_test[f'{CLASS_NAME}_stance'] = data_test['stance_label'].cat.codes
data_test[f'{CLASS_NAME}_argument'] = data_test['argument_label'].cat.codes

In [7]:
# Ready output data for the model
test_y_stance = to_categorical(data_test[f'{CLASS_NAME}_stance'])
test_y_argument = to_categorical(data_test[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
test_x = tokenizer(
    text=data_test['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

### 3.4.2 Architecture

In [8]:
# Build your model
input_ids = Input(shape=(256,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(256,), name='attention_mask', dtype='int32')
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[0]

# Add multi-head attention layer
attention_output = MultiHeadAttention(num_heads=3, key_dim=64)(bert_model, bert_model)

# Flatten the output from multi-head attention layer
flatten = Flatten()(attention_output)

# Apply dropout layer
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(flatten, training=False)

# Then build your model output
stance = Dense(units=len(data.stance_label.value_counts()), activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='stance')(pooled_output)
argument = Dense(units=len(data.argument_label.value_counts()), activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='argument')(pooled_output)
outputs = {'stance': stance, 'argument': argument}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  177853440   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 256,                      

### 3.4.3 Training

In [9]:
# Set an optimizer
optimizer = AdamW(
    learning_rate=1e-6,
    epsilon=1e-08,
    weight_decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'stance': CategoricalCrossentropy(from_logits = True), 'argument': CategoricalCrossentropy(from_logits = True)}
metric = {'stance': CategoricalAccuracy('accuracy'), 'argument': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Ready output data for the model
y_stance = to_categorical(data[f'{CLASS_NAME}_stance'])
y_argument = to_categorical(data[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

wandb.init(project="Argument mining", name = "Bert_attention_masks", tags = ["Ruberta_MHA", "RB"])
epochs = 8
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    y={'stance': y_stance, 'argument': y_argument},
    validation_data=({'input_ids': test_x['input_ids'][:8], 'attention_mask': test_x['attention_mask'][:8]}, 
                     {'stance': test_y_stance[:8], 'argument': test_y_argument[:8]}),
    batch_size=8,
    epochs=epochs, callbacks=[WandbCallback()])
wandb.finish()

wandb: Currently logged in as: benchm (darkdestiny). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/8


672/672 [==============================] - ETA: 0s - loss: 1.3001 - argument_loss: 0.5882 - stance_loss: 0.7119 - argument_accuracy: 0.8064 - stance_accuracy: 0.7331

wandb: Adding directory to artifact (/content/wandb/run-20230425_175705-g478norl/files/model-best)... Done. 12.3s


...

wandb: Adding directory to artifact (/content/wandb/run-20230425_175705-g478norl/files/model-best)... Done. 12.2s


672/672 [==============================] - 407s 606ms/step - loss: 0.5810 - argument_loss: 0.2379 - stance_loss: 0.3431 - argument_accuracy: 0.9172 - stance_accuracy: 0.8600 - val_loss: 1.0571 - val_argument_loss: 0.7568 - val_stance_loss: 0.3003 - val_argument_accuracy: 0.5000 - val_stance_accuracy: 0.8750
Epoch 8/8
672/672 [==============================] - ETA: 0s - loss: 0.5476 - argument_loss: 0.2241 - stance_loss: 0.3235 - argument_accuracy: 0.9205 - stance_accuracy: 0.8729

wandb: Adding directory to artifact (/content/wandb/run-20230425_175705-g478norl/files/model-best)... Done. 12.4s


672/672 [==============================] - 407s 606ms/step - loss: 0.5476 - argument_loss: 0.2241 - stance_loss: 0.3235 - argument_accuracy: 0.9205 - stance_accuracy: 0.8729 - val_loss: 0.9450 - val_argument_loss: 0.6915 - val_stance_loss: 0.2535 - val_argument_accuracy: 0.6250 - val_stance_accuracy: 0.8750


argument_accuracy,▁▆▇▇▇███
argument_loss,█▃▂▂▂▁▁▁
epoch,▁▂▃▄▅▆▇█
loss,█▄▃▃▂▂▁▁
stance_accuracy,▁▅▅▆▆▇▇█
stance_loss,█▄▄▃▂▂▁▁
val_argument_accuracy,██▁▁█▁▁█
val_argument_loss,█▇▅▅▂▂▂▁
val_loss,█▆▅▄▂▂▂▁
val_stance_accuracy,▁▅▆▆▆█▆▆
val_stance_loss,█▆▅▄▂▂▂▁


### 3.4.4 Prediction

In [12]:
CLASS_NAME = "masks"
test = pd.read_csv("./val_empty.tsv", sep='\t')
test.head()
test_d = test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]
for_pred = tokenizer(
    text=test_d['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
test_results = model.predict(x={'input_ids': for_pred['input_ids'], 'attention_mask': for_pred['attention_mask']})
test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1) - 1
test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1) - 1
test_d[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']].to_csv(f"./val_predict_MHA_{CLASS_NAME}.tsv", sep='\t', index=None)

45/45 [==============================] - 30s 566ms/step


<ipython-input-12-62ad68191405>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1) - 1
<ipython-input-12-62ad68191405>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1) - 1


In [13]:
CLASS_NAME = "masks"
test = pd.read_csv("./test-no_labels.tsv", sep='\t')
test.head()
test_d = test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]
for_pred = tokenizer(
    text=test_d['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
test_results = model.predict(x={'input_ids': for_pred['input_ids'], 'attention_mask': for_pred['attention_mask']})
test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1) - 1
test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1) - 1
test_d[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']].to_csv(f"./test_predict_MHA_{CLASS_NAME}.tsv", sep='\t', index=None)

44/44 [==============================] - 25s 570ms/step


<ipython-input-13-99c110209371>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1) - 1
<ipython-input-13-99c110209371>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1) - 1


## 3.5 vaccines

### 3.5.1 Preprocessing

In [14]:
CLASS_NAME = "vaccines"
# Import data from csv
whole_data = pd.read_csv('./train_all.tsv', sep='\t')

# Train_test_split
test_size = 0.2
data, data_test = train_test_split(whole_data, test_size=test_size)


#------------------------------------------------------------------------------------#
## Train
# Select required columns
data = data[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data['stance_label'] = pd.Categorical(data[f'{CLASS_NAME}_stance'])
data['argument_label'] = pd.Categorical(data[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data[f'{CLASS_NAME}_stance'] = data['stance_label'].cat.codes
data[f'{CLASS_NAME}_argument'] = data['argument_label'].cat.codes

#------------------------------------------------------------------------------------#
## Test
# Select required columns
data_test = data_test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data_test['stance_label'] = pd.Categorical(data_test[f'{CLASS_NAME}_stance'])
data_test['argument_label'] = pd.Categorical(data_test[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data_test[f'{CLASS_NAME}_stance'] = data_test['stance_label'].cat.codes
data_test[f'{CLASS_NAME}_argument'] = data_test['argument_label'].cat.codes
# Ready output data for the model
test_y_stance = to_categorical(data_test[f'{CLASS_NAME}_stance'])
test_y_argument = to_categorical(data_test[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
test_x = tokenizer(
    text=data_test['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

### 3.5.2 Architecture

In [15]:
# Build your model
input_ids = Input(shape=(256,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(256,), name='attention_mask', dtype='int32')
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[0]

# Add multi-head attention layer
attention_output = MultiHeadAttention(num_heads=3, key_dim=64)(bert_model, bert_model)

# Flatten the output from multi-head attention layer
flatten = Flatten()(attention_output)

# Apply dropout layer
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(flatten, training=False)

# Then build your model output
stance = Dense(units=len(data.stance_label.value_counts()), activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='stance')(pooled_output)
argument = Dense(units=len(data.argument_label.value_counts()), activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='argument')(pooled_output)
outputs = {'stance': stance, 'argument': argument}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  177853440   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 256,                      

### 3.5.3 Training

In [16]:
# Set an optimizer
optimizer = AdamW(
    learning_rate=1e-6,
    epsilon=1e-08,
    weight_decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'stance': CategoricalCrossentropy(from_logits = True), 'argument': CategoricalCrossentropy(from_logits = True)}
metric = {'stance': CategoricalAccuracy('accuracy'), 'argument': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Ready output data for the model
y_stance = to_categorical(data[f'{CLASS_NAME}_stance'])
y_argument = to_categorical(data[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

wandb.init(project="Argument mining", name = "Bert_attention_vaccines", tags = ["Ruberta_with_MHA", "RB"])
epochs = 8
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    y={'stance': y_stance, 'argument': y_argument},
    validation_data=({'input_ids': test_x['input_ids'][:8], 'attention_mask': test_x['attention_mask'][:8]}, 
                     {'stance': test_y_stance[:8], 'argument': test_y_argument[:8]}),
    batch_size=8,
    epochs=epochs, callbacks=[WandbCallback()])
wandb.finish()

Epoch 1/8


672/672 [==============================] - ETA: 0s - loss: 0.8025 - argument_loss: 0.3469 - stance_loss: 0.4556 - argument_accuracy: 0.8928 - stance_accuracy: 0.8357

wandb: Adding directory to artifact (/content/wandb/run-20230425_185602-r8qrp22p/files/model-best)... Done. 14.3s


672/672 [==============================] - 483s 626ms/step - loss: 0.8025 - argument_loss: 0.3469 - stance_loss: 0.4556 - argument_accuracy: 0.8928 - stance_accuracy: 0.8357 - val_loss: 0.2618 - val_argument_loss: 0.0193 - val_stance_loss: 0.2425 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 0.8750
Epoch 2/8
672/672 [==============================] - ETA: 0s - loss: 0.4750 - argument_loss: 0.2016 - stance_loss: 0.2734 - argument_accuracy: 0.9323 - stance_accuracy: 0.8785

wandb: Adding directory to artifact (/content/wandb/run-20230425_185602-r8qrp22p/files/model-best)... Done. 14.2s


...

wandb: Adding directory to artifact (/content/wandb/run-20230425_185602-r8qrp22p/files/model-best)... Done. 16.7s


672/672 [==============================] - 416s 618ms/step - loss: 0.3562 - argument_loss: 0.1468 - stance_loss: 0.2094 - argument_accuracy: 0.9464 - stance_accuracy: 0.9125 - val_loss: 0.0659 - val_argument_loss: 0.0543 - val_stance_loss: 0.0116 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000
Epoch 7/8
672/672 [==============================] - 340s 504ms/step - loss: 0.3315 - argument_loss: 0.1337 - stance_loss: 0.1978 - argument_accuracy: 0.9512 - stance_accuracy: 0.9205 - val_loss: 0.0957 - val_argument_loss: 0.0923 - val_stance_loss: 0.0034 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000
Epoch 8/8
672/672 [==============================] - ETA: 0s - loss: 0.3031 - argument_loss: 0.1222 - stance_loss: 0.1809 - argument_accuracy: 0.9555 - stance_accuracy: 0.9289

wandb: Adding directory to artifact (/content/wandb/run-20230425_185602-r8qrp22p/files/model-best)... Done. 21.4s


672/672 [==============================] - 431s 642ms/step - loss: 0.3031 - argument_loss: 0.1222 - stance_loss: 0.1809 - argument_accuracy: 0.9555 - stance_accuracy: 0.9289 - val_loss: 0.0427 - val_argument_loss: 0.0177 - val_stance_loss: 0.0250 - val_argument_accuracy: 1.0000 - val_stance_accuracy: 1.0000


argument_accuracy,▁▅▆▆▆▇██
argument_loss,█▃▃▂▂▂▁▁
epoch,▁▂▃▄▅▆▇█
loss,█▃▃▂▂▂▁▁
stance_accuracy,▁▄▅▅▆▇▇█
stance_loss,█▃▃▂▂▂▁▁
val_argument_accuracy,▁▁▁▁▁▁▁▁
val_argument_loss,▁▂▁▂▃▅█▁
val_loss,█▅▆▃▂▂▃▁
val_stance_accuracy,▁▁▁█████
val_stance_loss,█▅▆▃▂▁▁▂


In [17]:
CLASS_NAME = "vaccines"
test = pd.read_csv("./val_empty.tsv", sep='\t')
test.head()
test_d = test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]
for_pred = tokenizer(
    text=test_d['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
test_results = model.predict(x={'input_ids': for_pred['input_ids'], 'attention_mask': for_pred['attention_mask']})
test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1) - 1
test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1) - 1
test_d[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']].to_csv(f"./val_predict_MHA_{CLASS_NAME}.tsv", sep='\t', index=None)

45/45 [==============================] - 30s 566ms/step


<ipython-input-17-67c546eb642d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1) - 1
<ipython-input-17-67c546eb642d>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1) - 1


In [18]:
CLASS_NAME = "vaccines"
test = pd.read_csv("./test-no_labels.tsv", sep='\t')
test.head()
test_d = test[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]
for_pred = tokenizer(
    text=test_d['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
test_results = model.predict(x={'input_ids': for_pred['input_ids'], 'attention_mask': for_pred['attention_mask']})
test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1) - 1
test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1) - 1
test_d[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']].to_csv(f"./test_predict_MHA_{CLASS_NAME}.tsv", sep='\t', index=None)

44/44 [==============================] - 25s 575ms/step


<ipython-input-18-e571f3b82bb7>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d[f'{CLASS_NAME}_stance'] = test_results['stance'].argmax(axis=-1) - 1
<ipython-input-18-e571f3b82bb7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_d[f'{CLASS_NAME}_argument'] = test_results['argument'].argmax(axis=-1) - 1


## 3.6 Concatinate all files with results for masks, vaccines, quarantine

In [19]:
CLASS_NAME = "quarantine"
df1 = pd.read_csv(f"./val_predict_MHA_{CLASS_NAME}.tsv", sep='\t')
CLASS_NAME = "masks"
df2 = pd.read_csv(f"./val_predict_MHA_{CLASS_NAME}.tsv", sep='\t')
CLASS_NAME = "vaccines"
df3 = pd.read_csv(f"./val_predict_MHA_{CLASS_NAME}.tsv", sep='\t')


result = pd.merge(df1, df2, on="text")
result = pd.merge(result, df3, on="text")
result.to_csv("./val_predict_concat_MHA.tsv", sep='\t', index=None)

In [20]:
CLASS_NAME = "quarantine"
df1 = pd.read_csv(f"./test_predict_MHA_{CLASS_NAME}.tsv", sep='\t')
CLASS_NAME = "masks"
df2 = pd.read_csv(f"./test_predict_MHA_{CLASS_NAME}.tsv", sep='\t')
CLASS_NAME = "vaccines"
df3 = pd.read_csv(f"./test_predict_MHA_{CLASS_NAME}.tsv", sep='\t')


result = pd.merge(df1, df2, on="text")
result = pd.merge(result, df3, on="text")
result.to_csv("./test_predict_concat_MHA.tsv", sep='\t', index=None)

In [21]:
!zip val_predict_concat.zip val_predict_concat_MHA.tsv

  adding: val_predict_concat_MHA.tsv (deflated 73%)


In [22]:
!zip test_predict_concat.zip test_predict_concat_MHA.tsv

  adding: test_predict_concat_MHA.tsv (deflated 73%)
